# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

Description of project: Cournot duopoly

Equations:
Aggregate demand
Individual profit functions (used to find aggregate supply)

Analytical solution:
Find individual response functions

Grid search:
Try different outputs and check that supply equates demand.

Extensions:
Add more periods -> Easier to collude (påkrævet)
Vary delta -> see how collusion is harder to sustain
Vary homogeniety of products
Vary costs - draw from distribution (påkrævet)
Entry/exit 
Simulate model over time - plot quantity/price for different shocks to parameters (Corona-stød?)

Sjove plot (påkrævet)
Set up as class?

In [173]:
import numpy as np
import sympy as sm

from scipy.stats import norm
from scipy import optimize
from numpy import array
import matplotlib.pyplot as plt

%matplotlib inline
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Demand and cost

In [174]:
# Importing necessary libraries
import numpy as np
import scipy.optimize as optimize
import sympy as sm

# Defining symbolic variables for the optimization problem
x = sm.symbols('x')  # Symbolic variable representing x
x_rest = sm.symbols('x_rest')  # Symbolic variable representing x_rest
c=sm.symbols('c')

# Defining the objective function
#penalty = 100  # choose a suitable penalty value
objective = (1 - x - x_rest - c) * x
objective_lambd=sm.lambdify(args=(x,x_rest,c),expr=objective)

# Taking the first derivative of the objective function w.r.t x
obj_dif = sm.diff(objective, x)
# Converting the symbolic expression for the derivative into a callable function
best = sm.lambdify(args=(x, x_rest,c), expr=obj_dif)

# Taking the second derivative of the objective function w.r.t x
best_dif = sm.diff(obj_dif, x)

# Converting the symbolic expression for the second derivative into a callable function
jac_x = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Taking the second derivative of the objective function w.r.t x_rest
best_dif = sm.diff(obj_dif, x_rest)

# Converting the symbolic expression for the second derivative into a callable function
jac_x_rest = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Define the constraint that x should be greater than or equal to 0
#constraint = sm.Max(x, 0)

# Modify the objective to include the constraint
#modified_objective = sm.Piecewise((penalty*(-x), x < 0), (objective_orig*constraint, True))

# Convert the modified objective to a callable function
#modified_objective_lambd = sm.lambdify(args=(x, x_rest, c), expr=modified_objective)

In [175]:
# Defining the function to be optimized
def h(x):
    y = np.zeros(N)
    for i in range(N):
        # Evaluating the first derivative of the objective function at x[i]
        #c_rand = norm.rvs(loc=0, scale=0.001)
        y[i] = best(x[i], sum(x) - x[i], c_vec[i])
    return y

# Defining the Jacobian of the function to be optimized
def hp(x):
    y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if j == i:
                # Evaluating the second derivative of the objective function w.r.t x[i]
                y[i,j] = jac_x(x[i], sum(x) - x[i])
            else:
                # Evaluating the second derivative of the objective function w.r.t x_rest[i]
                y[i,j] = jac_x_rest(x[i], sum(x) - x[i])
    return y

In [250]:
#Initial løsning
# Setting up the parameters for the optimization problem
N = 10  # Number of variables

# Setting up the initial values for x_rest
x_rest = np.zeros(N)

np.random.seed(2000)
c_vec = np.random.normal(loc=0,scale=0.50,size=N)
c_vec=c_vec**8

# Setting up the initial values for x
x0 = np.array(x_rest)

# Solving the optimization problem using scipy.optimize.root() function
result = optimize.root(h, x0, jac=hp)

profit=objective_lambd(result.x,np.sum(result.x)-result.x,c_vec)
# Printing the results
print(result)
print('\nx =', result.x[0:10], '\nh(x) =', h(result.x)[0:10], '\nsum(x) =', sum(result.x), '\nmarginal cost=',c_vec[0:7],'\nprofit=', profit[0:10])

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
       x: [ 2.356e-01 -9.866e-02 -9.571e-01  5.590e-01  5.589e-01
           -2.092e+00  5.590e-01  5.587e-01  5.589e-01  5.589e-01]
    nfev: 6
    njev: 1
    fjac: [[-5.384e-01 -2.710e-01 ... -2.848e-01 -2.848e-01]
           [ 4.091e-01 -8.724e-01 ...  5.156e-02  5.156e-02]
           ...
           [ 1.338e-01  1.479e-02 ... -9.436e-01 -2.529e-02]
           [ 1.143e-01  1.263e-02 ...  1.240e-01 -9.514e-01]]
       r: [ 3.944e+00  1.889e+00 ...  1.705e-01  1.075e+00]
     qtf: [-5.879e-16  5.792e-17 -1.281e-16 -8.431e-17 -6.204e-17
           -1.258e-16  8.841e-17 -5.568e-17 -4.589e-17 -3.918e-17]

x = [ 0.23563928 -0.09865545 -0.957052    0.55896013  0.55890795 -2.09227514
  0.55896013  0.5587318   0.55887398  0.55894919] 
h(x) = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
sum(x) = 0.4410398699888224

In [246]:
#Anden del af løsningen (som skal køres k gange)

nonneg_x=result.x>=0
varlist_new=["x_rest", "c_vec", "x0"]
for var_name in varlist_new:
    var = globals()[var_name]
    globals()[var_name+'_new'] = var[nonneg_x]

N_new=sum(nonneg_x)
N_new

# Setting up the parameters for the optimization problem
#c = 0.001  # Constant value for the objective function
N = N_new # Number of variables

# Setting up the initial values for x_rest
x_rest = x_rest_new

np.random.seed(2000)
c_vec = c_vec_new

# Setting up the initial values for x
x0 = x0_new

# Solving the optimization problem using scipy.optimize.root() function
result = optimize.root(h, x0_new, jac=hp)

profit=objective_lambd(result.x,np.sum(result.x)-result.x,c_vec)
# Printing the results
print(result)
print('\nx =', result.x[0:10], '\nh(x) =', h(result.x)[0:10], '\nsum(x) =', sum(result.x), '\nmarginal cost=',c_vec[0:7],'\nprofit=', profit[0:10])

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
       x: [ 1.429e-01  1.429e-01  1.429e-01  1.427e-01  1.428e-01
            1.429e-01]
    nfev: 4
    njev: 1
    fjac: [[-7.559e-01 -3.779e-01 ...  9.218e-03  9.218e-03]
           [ 3.219e-01 -5.312e-01 ... -5.498e-01 -5.498e-01]
           ...
           [-4.032e-01  2.643e-01 ... -7.650e-01  2.065e-01]
           [-3.812e-01  2.498e-01 ...  2.498e-01 -7.778e-01]]
       r: [ 2.646e+00  2.246e+00 ...  5.774e-02  1.028e+00]
     qtf: [ 1.678e-16 -7.148e-17 -2.467e-17 -1.538e-17  8.953e-17
            8.464e-17]

x = [0.14291108 0.14285891 0.14291108 0.14268276 0.14282493 0.14290015] 
h(x) = [0. 0. 0. 0. 0. 0.] 
sum(x) = 0.8570889148316649 
marginal cost= [9.44495240e-10 5.21765403e-05 4.91579878e-10 2.28328866e-04
 8.61513706e-05 1.09379658e-05] 
profit= [0.02042358 0.02040867 0.02042358 0.02035837 0.02039896 0.02042045]


In [285]:
#Initial løsning
# Setting up the parameters for the optimization problem
N = 250  # Number of variables

# Setting up the initial values for x_rest
x_rest = np.zeros(N)

np.random.seed(2000)
c_vec = np.random.normal(loc=0,scale=0.50,size=N)
c_vec=c_vec**8

# Setting up the initial values for x
x0 = np.array(x_rest)

# Solving the optimization problem using scipy.optimize.root() function
result = optimize.root(h, x0, jac=hp)

nonneg_x=result.x>=0
varlist_new=["x_rest", "c_vec", "x0"]
for var_name in varlist_new:
    var = globals()[var_name]
    globals()[var_name+'_new'] = var[nonneg_x]

N_new=sum(nonneg_x)
while not all(nonneg_x):
    # Setting up the parameters for the optimization problem
    #c = 0.001  # Constant value for the objective function
    N = N_new # Number of variables

    # Setting up the initial values for x_rest
    x_rest = x_rest_new

    np.random.seed(2000)
    c_vec = c_vec_new

    # Setting up the initial values for x
    x0 = x0_new

    # Solving the optimization problem using scipy.optimize.root() function
    result = optimize.root(h, x0_new, jac=hp)

    nonneg_x=result.x>=0
    varlist_new=["x_rest", "c_vec", "x0"]
    for var_name in varlist_new:
        var = globals()[var_name]
        globals()[var_name+'_new'] = var[nonneg_x]

    N_new=sum(nonneg_x)

profit=objective_lambd(result.x,np.sum(result.x)-result.x,c_vec)
# Printing the results
print(result)
print('\nx =', result.x[0:10], '\nh(x) =', h(result.x)[0:10], '\nsum(x) =', sum(result.x), '\nmarginal cost=',c_vec[0:7],'\nprofit=', profit[0:10],'\nN_firms =',N)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  1.110e-16 ...  1.110e-16  1.110e-16]
       x: [ 5.957e-03  5.905e-03 ...  5.943e-03  3.863e-03]
    nfev: 7
    njev: 1
    fjac: [[-1.997e-01 -7.533e-02 ... -7.533e-02 -7.533e-02]
           [ 6.274e-01 -7.709e-01 ... -8.114e-03 -8.114e-03]
           ...
           [-5.309e-03 -6.594e-03 ...  9.970e-01  4.494e-04]
           [ 5.272e-03  6.548e-03 ...  6.460e-03 -9.970e-01]]
       r: [ 8.043e+00  6.700e+00 ... -6.954e-03  1.003e+00]
     qtf: [ 9.239e-15  5.761e-16 ...  1.847e-17 -1.834e-17]

x = [0.00595725 0.00590508 0.00595725 0.00572893 0.0058711  0.00594632
 0.00525515 0.00595706 0.00595723 0.00595725] 
h(x) = [0.00000000e+00 1.11022302e-16 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11022302e-16
 0.00000000e+00 1.11022302e-16] 
sum(x) = 0.9940427450405963 
marginal cost= [9.44495240e-10 5.21765403e-05 4.91579878e-10 2.28328866e-04
 8.61513706e-05 1.09379658e-0

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 